# Mechanizmy uwagi i transformatory

Jednym z głównych ograniczeń sieci rekurencyjnych jest to, że wszystkie słowa w sekwencji mają taki sam wpływ na wynik. Powoduje to suboptymalne działanie standardowych modeli LSTM typu encoder-decoder w zadaniach sekwencja-do-sekwencji, takich jak rozpoznawanie nazwanych jednostek (Named Entity Recognition) czy tłumaczenie maszynowe. W rzeczywistości konkretne słowa w sekwencji wejściowej często mają większy wpływ na wyniki sekwencyjne niż inne.

Rozważmy model sekwencja-do-sekwencji, taki jak tłumaczenie maszynowe. Jest on realizowany za pomocą dwóch sieci rekurencyjnych, gdzie jedna sieć (**encoder**) kompresuje sekwencję wejściową do stanu ukrytego, a druga, **decoder**, rozwija ten stan ukryty w przetłumaczony wynik. Problem z tym podejściem polega na tym, że końcowy stan sieci ma trudności z zapamiętaniem początku zdania, co prowadzi do niskiej jakości modelu w przypadku długich zdań.

**Mechanizmy uwagi** umożliwiają ważenie kontekstowego wpływu każdego wektora wejściowego na każdą prognozę wyjściową RNN. Realizuje się to poprzez tworzenie skrótów między stanami pośrednimi RNN wejściowego a RNN wyjściowego. W ten sposób, generując symbol wyjściowy $y_t$, uwzględniamy wszystkie stany ukryte wejścia $h_i$, z różnymi współczynnikami wagowymi $\alpha_{t,i}$.

![Obraz przedstawiający model encoder/decoder z warstwą uwagi addytywnej](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Model encoder-decoder z mechanizmem uwagi addytywnej w [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cytowany z [tego wpisu na blogu](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Macierz uwagi $\{\alpha_{i,j}\}$ reprezentuje stopień, w jakim określone słowa wejściowe wpływają na generowanie danego słowa w sekwencji wyjściowej. Poniżej znajduje się przykład takiej macierzy:

![Obraz przedstawiający przykładowe dopasowanie znalezione przez RNNsearch-50, zaczerpnięty z Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Rysunek zaczerpnięty z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Rys.3)*

Mechanizmy uwagi są odpowiedzialne za dużą część obecnych lub bliskich obecnym stanom sztuki osiągnięć w przetwarzaniu języka naturalnego. Dodanie mechanizmu uwagi znacznie jednak zwiększa liczbę parametrów modelu, co prowadzi do problemów ze skalowaniem w przypadku RNN. Kluczowym ograniczeniem skalowania RNN jest to, że rekurencyjny charakter modeli utrudnia grupowanie i równoległe trenowanie. W RNN każdy element sekwencji musi być przetwarzany w kolejności sekwencyjnej, co oznacza, że nie można go łatwo zrównoleglić.

Zastosowanie mechanizmów uwagi w połączeniu z tym ograniczeniem doprowadziło do powstania obecnych modeli Transformer, które znamy i używamy dzisiaj, takich jak BERT czy OpenGPT3.

## Modele Transformer

Zamiast przekazywać kontekst każdej poprzedniej prognozy do kolejnego kroku ewaluacji, **modele Transformer** wykorzystują **kodowania pozycyjne** i mechanizmy uwagi, aby uchwycić kontekst danego wejścia w określonym oknie tekstowym. Poniższy obraz pokazuje, jak kodowania pozycyjne z uwagą mogą uchwycić kontekst w danym oknie.

![Animowany GIF pokazujący, jak przeprowadzane są ewaluacje w modelach Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Ponieważ każda pozycja wejściowa jest mapowana niezależnie na każdą pozycję wyjściową, transformatory mogą być lepiej zrównoleglane niż RNN, co umożliwia tworzenie większych i bardziej ekspresywnych modeli językowych. Każda głowica uwagi może być używana do nauki różnych relacji między słowami, co poprawia zadania przetwarzania języka naturalnego.

**BERT** (Bidirectional Encoder Representations from Transformers) to bardzo duża, wielowarstwowa sieć Transformer z 12 warstwami dla *BERT-base* i 24 dla *BERT-large*. Model jest najpierw wstępnie trenowany na dużym korpusie danych tekstowych (Wikipedia + książki) za pomocą uczenia nienadzorowanego (przewidywanie zamaskowanych słów w zdaniu). Podczas wstępnego trenowania model przyswaja znaczący poziom zrozumienia języka, który można następnie wykorzystać z innymi zbiorami danych za pomocą dostrajania. Ten proces nazywa się **uczeniem transferowym**.

![obrazek z http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Istnieje wiele wariantów architektur Transformer, w tym BERT, DistilBERT, BigBird, OpenGPT3 i inne, które można dostrajać. Pakiet [HuggingFace](https://github.com/huggingface/) udostępnia repozytorium do trenowania wielu z tych architektur z wykorzystaniem PyTorch.

## Wykorzystanie BERT do klasyfikacji tekstu

Zobaczmy, jak możemy wykorzystać wstępnie wytrenowany model BERT do rozwiązania naszego tradycyjnego zadania: klasyfikacji sekwencji. Będziemy klasyfikować nasz oryginalny zbiór danych AG News.

Najpierw załadujmy bibliotekę HuggingFace i nasz zbiór danych:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Ponieważ będziemy korzystać z wstępnie wytrenowanego modelu BERT, musimy użyć odpowiedniego tokenizatora. Najpierw załadujemy tokenizator powiązany z wstępnie wytrenowanym modelem BERT.

Biblioteka HuggingFace zawiera repozytorium wstępnie wytrenowanych modeli, z których można korzystać, podając ich nazwy jako argumenty do funkcji `from_pretrained`. Wszystkie wymagane pliki binarne dla modelu zostaną automatycznie pobrane.

Jednakże, w niektórych przypadkach konieczne będzie załadowanie własnych modeli. W takim przypadku można wskazać katalog zawierający wszystkie odpowiednie pliki, w tym parametry dla tokenizatora, plik `config.json` z parametrami modelu, wagi binarne itp.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Obiekt `tokenizer` zawiera funkcję `encode`, która może być bezpośrednio użyta do kodowania tekstu:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Następnie stwórzmy iteratory, które będziemy używać podczas treningu do uzyskiwania dostępu do danych. Ponieważ BERT używa własnej funkcji kodowania, musielibyśmy zdefiniować funkcję dopełniania podobną do `padify`, którą zdefiniowaliśmy wcześniej:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

W naszym przypadku będziemy używać wstępnie wytrenowanego modelu BERT o nazwie `bert-base-uncased`. Załadujmy model za pomocą pakietu `BertForSequenceClassfication`. To zapewnia, że nasz model ma już wymaganą architekturę do klasyfikacji, w tym końcowy klasyfikator. Zobaczysz komunikat ostrzegawczy informujący, że wagi końcowego klasyfikatora nie są zainicjalizowane i model wymaga wstępnego treningu - to jest całkowicie w porządku, ponieważ dokładnie to zamierzamy zrobić!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Teraz możemy rozpocząć trening! Ponieważ BERT jest już wstępnie wytrenowany, chcemy zacząć od dość małej wartości współczynnika uczenia, aby nie zniszczyć początkowych wag.

Całą ciężką pracę wykonuje model `BertForSequenceClassification`. Gdy wywołujemy model na danych treningowych, zwraca on zarówno stratę, jak i wynik sieci dla wejściowej mini-paczki. Stratę wykorzystujemy do optymalizacji parametrów (`loss.backward()` wykonuje propagację wsteczną), a `out` do obliczania dokładności treningu, porównując uzyskane etykiety `labs` (obliczone za pomocą `argmax`) z oczekiwanymi `labels`.

Aby kontrolować proces, akumulujemy stratę i dokładność przez kilka iteracji i wypisujemy je co `report_freq` cykli treningowych.

Ten trening prawdopodobnie zajmie dość dużo czasu, więc ograniczamy liczbę iteracji.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Możesz zauważyć (szczególnie jeśli zwiększysz liczbę iteracji i poczekasz wystarczająco długo), że klasyfikacja za pomocą BERT daje nam całkiem dobrą dokładność! Dzieje się tak, ponieważ BERT już bardzo dobrze rozumie strukturę języka, a my musimy jedynie dostroić końcowy klasyfikator. Jednakże, ponieważ BERT to duży model, cały proces treningu zajmuje dużo czasu i wymaga dużej mocy obliczeniowej! (GPU, a najlepiej więcej niż jednego).

> **Note:** W naszym przykładzie używamy jednego z najmniejszych wstępnie wytrenowanych modeli BERT. Istnieją większe modele, które prawdopodobnie przyniosą lepsze wyniki.


## Ocena wydajności modelu

Teraz możemy ocenić wydajność naszego modelu na zestawie testowym. Pętla oceny jest bardzo podobna do pętli treningowej, ale nie możemy zapomnieć o przełączeniu modelu w tryb oceny, wywołując `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Kluczowe informacje

W tej jednostce zobaczyliśmy, jak łatwo można wykorzystać wstępnie wytrenowany model językowy z biblioteki **transformers** i dostosować go do naszego zadania klasyfikacji tekstu. Podobnie, modele BERT mogą być używane do ekstrakcji encji, odpowiadania na pytania i innych zadań związanych z NLP.

Modele transformerów reprezentują obecny stan wiedzy w dziedzinie NLP i w większości przypadków powinny być pierwszym rozwiązaniem, od którego zaczynasz eksperymenty przy implementacji niestandardowych rozwiązań NLP. Jednak zrozumienie podstawowych zasad działania rekurencyjnych sieci neuronowych, omówionych w tym module, jest niezwykle ważne, jeśli chcesz budować zaawansowane modele neuronowe.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
